In [ ]:
#车牌图像训练

In [26]:
import cv2
import os
import os.path
import math
import random
import sys
import numpy as np
import matplotlib.cm as cm
import matplotlib.mlab as mlab
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torch.nn.functional as F

In [34]:
conv=nn.Sequential(
            nn.Conv2d(1,64,kernel_size=3,padding=1), #layer1, inputs single channel,256*128
            nn.ReLU(inplace=True),
            nn.Conv2d(64,64,kernel_size=3,padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2,stride=2),
            nn.Conv2d(64,128,kernel_size=3,padding=1), #layer2 inputs 64 channel,128*64
            nn.ReLU(inplace=True),
            nn.Conv2d(128,128,kernel_size=3,padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2,stride=2),
            nn.Conv2d(128,256,kernel_size=3,padding=1), #layer3 inputs 128 channel,64*32
            nn.ReLU(inplace=True),
            nn.Conv2d(256,256,kernel_size=3,padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256,256,kernel_size=3,padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2,stride=2),
            nn.Conv2d(256,512,kernel_size=3,padding=1), #layer4 inputs 256 channel,32*16
            nn.ReLU(inplace=True),
            nn.Conv2d(512,512,kernel_size=3,padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(512,512,kernel_size=3,padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2,stride=2),
            nn.Conv2d(512,512,kernel_size=3,padding=1), #layer5 inputs 512 channel,16*8
            nn.ReLU(inplace=True),
            nn.Conv2d(512,512,kernel_size=3,padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(512,512,kernel_size=3,padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2,stride=2)
    )

class vgg16train(nn.Module):
    def __init__(self):
        super(vgg16train,self).__init__()
        self.features=conv
        self.classifier=nn.Sequential(
            nn.Dropout(),
            nn.Linear(512 * 8 * 4, 4096),                             #connect: layter: inputs 512, 8*4
            nn.ReLU(True),
            nn.Dropout(),
            nn.Linear(4096, 2048),
            nn.ReLU(True),
            nn.Dropout(),
            nn.Linear(2048, 2)
        )
        #initialize_weights
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
                m.weight.data.normal_(0, math.sqrt(2. / n))
                if m.bias is not None:
                    m.bias.data.zero_()
            elif isinstance(m, nn.BatchNorm2d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()
            elif isinstance(m, nn.Linear):
                n = m.weight.size(1)
                m.weight.data.normal_(0, 0.01)
                m.bias.data.zero_()
    def forward(self,x):
        x=self.features(x)
        x=x.view(x.size(0),-1)
        x=self.classifier(x)
        return x

class npModel(nn.Module):
    def __init__(self):
        super(npModel, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=3,padding=1)   
        self.conv2 = nn.Conv2d(10, 20, kernel_size=3,padding=1) 
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(20*8*16, 100)   #16*32*20
        self.fc2 = nn.Linear(100, 2)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))                                #32*64
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))   #16*32
        x = x.view(-1, 20*8*16)      
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x)        

In [35]:
class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count    
        
def accuracy(output, target, topk=(1,)):
    """Computes the precision@k for the specified values of k"""
    maxk = max(topk)
    batch_size = target.size(0)

    _, pred = output.topk(maxk, 1, True, True)   #pred是索引/位置指示
    pred = pred.t()                                                 #转置
    correct = pred.eq(target.view(1, -1).expand_as(pred))

    res = []
    for k in topk:
        correct_k = correct[:k].view(-1).float().sum(0)
        res.append(correct_k.mul_(100.0 / batch_size))
    return res        

In [ ]:
#get back picture 生成反例图片/背景图片
OUTPUT_SHAPE=(128,256)
SRCPICDIR='/home/wang/git/deep-anpr/bgs'
DSTPICDIR='/home/wang/git/nppic/nproot/back'
SRC_NUM_PIC=10000
DST_NUM_PIC=500
def generate_bg(num_bg_img):
    found=False
    while not found :
        fname="{}/{:08d}.jpg".format(SRCPICDIR,random.randint(0, SRC_NUM_PIC - 1))
        bg=cv2.imread(fname,cv2.CV_LOAD_IMAGE_GRAYSCALE)
        if(bg.shape[0]>=OUTPUT_SHAPE[0]  and bg.shape[1]>=OUTPUT_SHAPE[1]):
            found=True
    x=random.randint(0,bg.shape[1]-OUTPUT_SHAPE[1])
    y=random.randint(0,bg.shape[0]-OUTPUT_SHAPE[0])
    return bg[y:y+OUTPUT_SHAPE[0],x:x+OUTPUT_SHAPE[1]]
    
if os.path.exists(DSTPICDIR):
    raise IOError,DSTPICDIR+" has been exists" 
os.mkdir(DSTPICDIR)            
i=0
while i<600:
    #bg=generate_bg(SRC_NUM_PIC)
    fname=os.path.join(DSTPICDIR,"{:4d}.jpg".format(random.randint(0,9999)))
    #cv2.imwrite(fname,generate_bg(SRC_NUM_PIC))
    i=i+1

In [33]:
#mean and variance 得到均值和方差
TRAINDIR='/home/wang/git/nppic/nproot'
f=[]
for parent,dirnames,filenames in os.walk(TRAINDIR):
     for i,fname in enumerate(filenames):
        f.append(os.path.join(parent,fname))
img=np.ndarray((len(f),128,256))
for i,fname in enumerate(f):
    img[i]=cv2.imread(os.path.join(TRAINDIR,fname),cv2.CV_LOAD_IMAGE_GRAYSCALE)
mean=np.mean(img)
var=np.var(img)
print mean,var
#不除以255：mean/var:107.505113917 3847.16795699
#除以255，mean/var:0.42312632171/0.0598781834145


107.505113917 3847.16795699


In [22]:
#def make_scaled_ims(im, min_shape):
#    ratio = 1. / 2 ** 0.5
#        yield cv2.resize(im, (shape[1], shape[0]))
        

In [36]:
def gray_loader(path):#input 128*256
    im=cv2.imread(path,cv2.CV_LOAD_IMAGE_GRAYSCALE)
    return im


In [91]:
batch_size=10
data_transform=transforms.Compose([transforms.ToTensor(),
                              transforms.Normalize((107.5051,), (3847.1679,)),
                             ])
npset = datasets.ImageFolder(root='/home/wang/git/nppic/nproot', 
                             transform=data_transform,loader=gray_loader)
nploader = torch.utils.data.DataLoader(npset, batch_size=batch_size, shuffle=True, 
                                       num_workers=1)  #train
npvalset=datasets.ImageFolder(root='/home/wang/git/nppic/npval', 
                              transform=data_transform,loader=gray_loader)
npvalloader=torch.utils.data.DataLoader(npvalset, batch_size=batch_size, shuffle=True, 
                                        num_workers=1) #validate
print 'over'

over


In [46]:
model=vgg16train()
model.cuda()
criterion = nn.CrossEntropyLoss() # use a Classification Cross-Entropy loss
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
print 'over'

over


In [47]:
#Train
losses=AverageMeter()
model.train()
for epoch in range(20):
    losses.reset()
    for _, (datas,targets) in enumerate(nploader):
        datas=torch.unsqueeze(datas,1)
        datas_var,targets_var=torch.autograd.Variable(datas),torch.autograd.Variable(targets)
        datas_var=datas_var.cuda()
        targets_var=targets_var.cuda()
        
        optimizer.zero_grad()
        outputs_var = model(datas_var)
        loss = criterion(outputs_var, targets_var)
        loss.backward()        
        optimizer.step()
        #loss
        losses.update(loss.data[0],datas_var.data.size(0))  # datas size(0) is batch_size of input
        if losses.count % 500 == 0:
            print ('[%d, %4d] currnet los: %.5f, ave loss: %.5f' %(epoch+1,losses.count,loss.data[0], losses.avg)) 
 

[1,  500] currnet los: 0.68745, ave loss: 0.69291
[2,  500] currnet los: 0.69497, ave loss: 0.69179
[3,  500] currnet los: 0.67789, ave loss: 0.69171
[4,  500] currnet los: 0.70699, ave loss: 0.68679
[5,  500] currnet los: 0.67425, ave loss: 0.67894
[6,  500] currnet los: 0.47749, ave loss: 0.62289
[7,  500] currnet los: 0.00822, ave loss: 0.23472
[8,  500] currnet los: 0.00332, ave loss: 0.05390
[9,  500] currnet los: 0.11020, ave loss: 0.04216
[10,  500] currnet los: 0.05372, ave loss: 0.03468
[11,  500] currnet los: 0.00481, ave loss: 0.02191
[12,  500] currnet los: 0.03106, ave loss: 0.02313
[13,  500] currnet los: 0.00013, ave loss: 0.02183
[14,  500] currnet los: 0.03563, ave loss: 0.01895
[15,  500] currnet los: 0.01196, ave loss: 0.02631
[16,  500] currnet los: 0.00515, ave loss: 0.03786
[17,  500] currnet los: 0.01267, ave loss: 0.04407
[18,  500] currnet los: 0.01212, ave loss: 0.01058
[19,  500] currnet los: 0.00000, ave loss: 0.00254
[20,  500] currnet los: 0.00037, ave los

In [48]:
torch.save(model.state_dict(),'./plate.weight')

In [92]:
#validate
prediction=AverageMeter()
model.eval()
for _, (datas,targets) in enumerate(npvalloader):
    datas=torch.unsqueeze(datas,1)
    datas_var,targets_var=torch.autograd.Variable(datas),torch.autograd.Variable(targets)
    datas_var=datas_var.cuda()
    targets_var=targets_var.cuda()    
    outputs_var = model(datas_var)
    prec=accuracy(outputs_var.data,targets_var.data)   #40% will return 40
    prediction.update(prec[0].cpu().numpy()[0],datas_var.data.size(0))
    
#print(' accuracy is{val.avg:%.2f}'.format(val=prediction))
print ('accuracy is: %.2f' %(prediction.avg)) 


accuracy is: 100.00


In [54]:
net=vgg16train()
net.load_state_dict(torch.load('/home/wang/git/anpr/plate.weight'))
net.cuda()
print('over')

over


In [89]:
nptestset=datasets.ImageFolder(root='/home/wang/git/nppic/npr', 
                              transform=data_transform,loader=gray_loader)
nptestloader=torch.utils.data.DataLoader(npvalset, batch_size=1, shuffle=False, 
                                        num_workers=1) #validate
def find_classes(dir):
    classes = [d for d in os.listdir(dir) if os.path.isdir(os.path.join(dir, d))]
    classes.sort()
    class_to_idx = {classes[i]: i for i in range(len(classes))}
    return classes, class_to_idx
classes,class_to_idx=find_classes('/home/wang/git/nppic/npr')
print class_to_idx

{'plate': 1, 'back': 0}


In [90]:
nptest=iter(nptestloader)
datas,targets=nptest.next()
targets
#targets


 1
[torch.LongTensor of size 1]

In [84]:
datas=torch.unsqueeze(datas,1)
datas_var,targets_var=torch.autograd.Variable(datas),torch.autograd.Variable(targets)
datas_var=datas_var.cuda()
targets_var=targets_var.cuda()   
outputs_var=net(datas_var)
print('over')

over


In [85]:
outputs_var

Variable containing:
-10.3887   8.9537
[torch.cuda.FloatTensor of size 1x2 (GPU 0)]

In [86]:
classesc=['back','plate']
_,pred=torch.max(outputs_var,1)
print pred

Variable containing:
 1
[torch.cuda.LongTensor of size 1x1 (GPU 0)]



In [69]:
p=pred.cpu().data.numpy()
for i,idx in enumerate(p):
    print classesc[idx[0]]
#

Variable containing:
 1
[torch.cuda.LongTensor of size 1x1 (GPU 0)]

plate


In [ ]:
#mean and variance
TRAINDIR='/home/wang/git/nppic/nproot/plate'
f=[]
for parent,dirnames,filenames in os.walk(TRAINDIR):
    for filename in filenames:
        f.append(filename)
img=np.ndarray((len(f),128,256))
for i,fname in enumerate(f):
    img[i]=cv2.imread(os.path.join(TRAINDIR,fname),cv2.CV_LOAD_IMAGE_GRAYSCALE)

img=img/255  #[0,1]

mean=np.mean(img)   #-0.18262765251608459
var=np.var(img)             #0.21571192247129692
    